In [114]:
import openai
import pickle
%load_ext autoreload
%autoreload 2

from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import os
import sys
import requests
import json
import numpy as np
import pandas as pd
import pickle as pkl
sys.path.append('/sharespace/flock_testnet/')

from tqdm.notebook import tqdm
import ml_utils
import re
import warnings
warnings.filterwarnings('ignore')


pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 1000)
pd.set_option('display.max_rows', 1600)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
sec = ml_utils.load_json('../credentials/api_keys.json')
openai_key = sec['openai_key']
openai_org = sec['openai_org']

In [16]:
# read file
files = ml_utils.get_filefolder_info_df('../feature_data/task5/farcaster_files/')

In [17]:
files.columns

Index(['filename', 'ext', 'size_mb', 'last_modified', 'created', 'fullpath'], dtype='object')

In [46]:
def chat_with_gpt(prompt, model="gpt-4", temperature=0.7):    
    client = OpenAI(api_key=openai_key, organization=openai_org)
    messages = [
        {
            "role": "user", "content": prompt},
    ]
    kwargs = dict(model=model, messages=messages, max_tokens=500)    
    
    completion = client.chat.completions.create(**kwargs)
    res = completion.choices[0].message.content
    return res

In [251]:
PROMPT_QUESTION_TEMPLATE = """ Given input text: {input_text} 
as a professional technical document writer, you will provide a list of questsions regarding the key entity and activity descripted in the input text. 

Do the following for 1 to 10 iterations:
1. Identify 1-3 concise entities or technical activity in the input text,.
2. Ask a questsion about each of these entities or activities, typically asking for explanation or how certain things are done 

Respond in a JSON in the following format:
{{
    entity1 : text of question for entity 1  
    entity2 : text of question for entity 2
}}
"""

In [ ]:
# {[question1, question2, etc]


In [252]:
# instructions_assistant = "please answer in a technical and professional manner, identy key concepts that are of particular interest from a user aspect, and propose questsions that are core to the input text"

prompt = PROMPT_QUESTION_TEMPLATE

use_files = files#.head(3)
question_jsons = []
for i, row in tqdm(use_files.iterrows(), total = len(use_files)):
    row_filepath = row['fullpath']
    with open(row_filepath, 'r') as file:
        row_text = file.read()
    row_prompt = prompt.format(input_text=row_text)
    try:
        response = chat_with_gpt(row_prompt)
        question_jsons.append(response)
    except:
        pass

  0%|          | 0/108 [00:00<?, ?it/s]

In [159]:
len(question_jsons)

108

creating file ../feature_data/task5/facaster_question_v0.pkl


'../feature_data/task5/facaster_question_v0.pkl'

# post process

In [253]:



def extract_text_between_curly_braces(text):
  """
  Extracts all text between "{" and "}" pairs in a text using regular expressions.

  Args:
    text: The text to extract from.

  Returns:
    A list of all the extracted text snippets.
  """
  pattern = r"{([^}]*)}"  # Matches any character except } between { and }
  matches = re.findall(pattern, text)
  return matches
  # return data#json.dumps(data)


def remove_empty_lines(text):
  """
  Removes all empty lines from a piece of text.

  Args:
    text: The text to process.

  Returns:
    A string containing the text with all empty lines removed.
  """
  return "\n".join([line for line in text.splitlines() if line.strip()])



def convert_text_to_list(text):
  """
  Converts text in a specific format (question-answer pairs) to a list of dictionaries.

  Args:
    text: The text to convert.

  Returns:
    A list of dictionaries, where each dictionary represents a question-answer pair.
  """
  data_list = []
  for line in text.splitlines():
    if not line:  # Skip empty lines
      continue
    try:
        key, value = line.split(" : ", 1)  # Split by ": " delimiter, keeping only the first occurrence
        # data_list.append({"entity": key.strip().strip('"'), "question": value.strip().strip('"')})
        key = key.strip().strip('"')
        value = value.strip().strip(',').strip('"')
        data_list.append({"entity": key, "question": value})

    except:
        pass
  return data_list

In [254]:
question_items = []
for item in question_jsons:
    res = extract_text_between_curly_braces(item)
    for ritem in res:    
        question_items.append(ritem)

question_txt = "\n".join(question_items)
# question_txt = remove_empty_lines(question_txt)
qd = convert_text_to_list(question_txt)
qdf = pd.DataFrame.from_dict(qd)

In [255]:
qdf.shape

(642, 2)

In [257]:
qdf

,entity,question
0,Farcaster Network Messages,What format do these messages have to be in for interaction within the Farcaster network?
1,Peer-to-peer Network Nodes (Hubs),How is the peer-to-peer network connected and how does a message propagate within the network?
2,Account Keys,What security measures are in place while generating the keys? Can these keys be regenerated in case of a loss?
3,Message Types,Can these message types be customized and how is the type of a message verified within the network?
4,Storage Rental,How is the cost for storage rental determined and how are users notified of their nearing storage limit?
5,Storage Registry,How is the amount of storage usage tracked and validated in the storage registry?
6,Message Deletion,What happens to the message space once a message is deleted and how is the deletion verified within the network?
7,Timestamps on Messages,Why are timestamps on messages unverified? Does this not lead to potential security risks?
8,Farcaster Epoch,What is the importance of the Farcaster Epoch and how was the specific date and time decided?
9,Message Specifications,What do the message specifications entail and are they bound by any rules or limitations?


In [258]:
ml_utils.pickle_data('../feature_data/task5/facaster_question.pkl', question_jsons, timestamp=True)

creating file ../feature_data/task5/facaster_question_20240615.pkl


'../feature_data/task5/facaster_question_20240615.pkl'

In [199]:
columns = ['entity', 'question']
qdf = pd.DataFrame()

content=[]
for item in question_txt:
    item_dict = {}
    item_dict['entity'] = item

"Offchain ENS Names (Fnames)" : "How can one access and interpret Fname Registry to fetch details associated with offchain ENS Names (Fnames)?",
